# Juntando as Peças 🧩

## (Fontes de Dados 🚰 ➕ Seleção 🍕 ➕ Filtro 🍝) <sup>Alto Desempenho 🚀 🧞‍♂️</sup>

### Funções Auxiliares

In [1]:
import os, json, sqlite3, pandas as pd

def build_path(subfolder = 'raw'):
    folderpath = os.path.join(os.getcwd(), os.pardir, 
                              'project', 'data', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): 
        os.makedirs(folderpath)
    return folderpath

def get_estados_georreferenciamento(filename='estados_georreferenciamento.csv'):
    filepath = os.path.join(build_path(), filename)
    return pd.read_csv(filepath)

def get_municipios_georreferenciamento(filename='municipios_georreferenciamento.csv'):
    filepath = os.path.join(build_path(), filename)
    return pd.read_csv(filepath)

def get_covid_infections_and_deaths(filename='ALL_HIST_PAINEL_COVID.csv'):
    filepath = os.path.join(build_path(), filename)
    return pd.read_csv(filepath, sep=';', parse_dates = ['data'],\
                       dtype={'codmun': 'Int64', \
                              'codRegiaoSaude': 'Int64', \
                              'populacaoTCU2019': 'Int64', \
                              'casosAcumulado': 'Int64', \
                              'Recuperadosnovos': 'Int64', \
                              'emAcompanhamentoNovos': 'Int64', \
                              'interior/metropolitana': 'Int64'})

def get_estados_codigos(filename='estados_codigos.json'):
    filepath = os.path.join(build_path(), filename)
    with open(filepath) as jsonfile:
        return pd.json_normalize(json.load(jsonfile))

def get_estados_caracteristicas(filename='estados_caracteristicas.json'):
    filepath = os.path.join(build_path(), filename)
    with open(filepath) as jsonfile:
        return pd.json_normalize(json.load(jsonfile), \
                                 record_path='characteristics', \
                                 record_prefix='characteristics_', \
                                 meta='state',\
                                 meta_prefix='state_')

def get_estados_vacinacao(filename='estados_vacinacao.json'):
    filepath = os.path.join(build_path(), filename)
    with open(filepath) as jsonfile:
        df = pd.json_normalize(json.load(jsonfile)['Paciente_Estado']['buckets'], \
                               record_path=['Data_Aplicacao_Vacina', ['buckets']], \
                               meta=['key', 'doc_count'], \
                               record_prefix='vacinacao_', \
                               meta_prefix='estado_').convert_dtypes()
        df['vacinacao_key'] = pd.to_datetime(df['vacinacao_key'], unit='ms')
        return df

def get_municipios_vacinacao(filename='municipios_vacinacao.json'):
    filepath = os.path.join(build_path(), filename)
    with open(filepath) as jsonfile:
        df = pd.json_normalize(json.load(jsonfile)['Paciente_Municipio']['buckets'], \
                               record_path=['Data_Aplicacao_Vacina', ['buckets']], \
                               meta=['key', 'doc_count'], \
                               record_prefix='vacinacao_', \
                               meta_prefix='municipio_',)
        df['vacinacao_key'] = pd.to_datetime(df['vacinacao_key'], unit='ms')
        return df[df['municipio_key'] != ''].convert_dtypes()

def get_municipios_codigos(filename='municipios_codigos.db'):
    filepath = os.path.join(build_path(), filename)
    conn = sqlite3.connect(filepath)
    municipios_codigos = pd.read_sql_query('select * from ibge', conn)
    return municipios_codigos

def load_data():
    return {'estados_georreferenciamento': get_estados_georreferenciamento(),
            'municipios_georreferenciamento': get_municipios_georreferenciamento(),
            'infections_and_deaths': get_covid_infections_and_deaths(),
            'estados_codigos': get_estados_codigos(),
            'estados_caracteristicas': get_estados_caracteristicas(),
            'estados_vacinacao': get_estados_vacinacao(),
            'municipios_vacinacao': get_municipios_vacinacao(),
            'municipios_codigos': get_municipios_codigos()}

In [2]:
def clean_estados_georreferenciamento(df, filename='estados_georreferenciamento.json'):
    filepath = os.path.join(build_path('clean'), filename)
    df[['codigo_uf', 'latitude', 'longitude']].to_json(filepath)

def clean_municipios_georreferenciamento(df, filename='municipios_georreferenciamento.json'):
    filepath = os.path.join(build_path('clean'), filename)
    df[['codigo_ibge', 'latitude', 'longitude']].to_json(filepath)

def clean_covid_infections_and_deaths(df, filename='ALL_HIST_PAINEL_COVID.json'):
    filepath = os.path.join(build_path('clean'), filename)
    df[df['municipio'].notna()][['codmun', 'data', 'casosNovos', 'obitosNovos', 'populacaoTCU2019']].to_json(filepath)

def clean_estados_codigos(df, filename='estados_codigos.json'):
    filepath = os.path.join(build_path('clean'), filename)
    df[['id', 'sigla', 'nome']].to_json(filepath)

def clean_estados_caracteristicas(df, filename='estados_caracteristicas.json'):
    filepath = os.path.join(build_path('clean'), filename)
    df['characteristics_value'] = df['characteristics_value'] + ' ' + df['characteristics_measure']
    df[['state_state', 'characteristics_label', 'characteristics_value']].to_json(filepath)

def clean_estados_vacinacao(df, filename='estados_vacinacao.json'):
    filepath = os.path.join(build_path('clean'), filename)
    df['vacinacao_key'] = df['vacinacao_key'].dt.date
    df[['estado_key', 'vacinacao_key', 'vacinacao_doc_count']].to_json(filepath)

def clean_municipios_vacinacao(df, filename='municipios_vacinacao.json'):
    filepath = os.path.join(build_path('clean'), filename)
    df[['municipio_key', 'vacinacao_key', 'vacinacao_doc_count']].to_json(filepath)

def clean_municipios_codigos(df, filename='municipios_codigos.json'):
    filepath = os.path.join(build_path('clean'), filename)
    df[['id', 'city']].to_json(filepath)

### Código

In [3]:
data = load_data()

In [4]:
clean_estados_georreferenciamento(data['estados_georreferenciamento'])
clean_municipios_georreferenciamento(data['municipios_georreferenciamento'])
clean_estados_caracteristicas(data['estados_caracteristicas'])
clean_estados_codigos(data['estados_codigos'])
clean_municipios_codigos(data['municipios_codigos'])
clean_estados_vacinacao(data['estados_vacinacao'])
clean_municipios_vacinacao(data['municipios_vacinacao'])
clean_covid_infections_and_deaths(data['infections_and_deaths'])